# Worksheet 05 - XML

Your Name:  
Your Class:   INST 447  
Your Section: 0101 (MWF) or 0102 (TTh)  
Your favorite color:  

Citations:
XML Data Sets: http://aiweb.cs.washington.edu/research/projects/xmltk/xmldata/www/repository.html


In [1]:
# standard includes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline

In [2]:
# import the lxml parser
from lxml import etree

Validation works the same with a DTD as it does with an XML Schema

In [3]:
dtd = etree.DTD('ebay.dtd')
ebay = etree.parse('ebay.xml') 

### Q1 - Validate IT!

How do we tell if it is valid? (The dtd object works similar to the schema object...)

In [4]:
# Hint: The dtd object works just like the schema object


Well now we can look inside!

In [5]:
node_count = 0
for listing in ebay.findall('//listing'):
    print("<listing>")
    for child in listing.getchildren():
        print(" ", child.tag, child.attrib)
    print("</listing>")
    node_count += 1
print("\nNumber of nodes: {0}".format(node_count))

<listing>
  seller_info {}
  payment_types {}
  shipping_info {}
  buyer_protection_info {}
  auction_info {}
  bid_history {}
  item_info {}
</listing>
<listing>
  seller_info {}
  payment_types {}
  shipping_info {}
  buyer_protection_info {}
  auction_info {}
  bid_history {}
  item_info {}
</listing>
<listing>
  seller_info {}
  payment_types {}
  shipping_info {}
  buyer_protection_info {}
  auction_info {}
  bid_history {}
  item_info {}
</listing>
<listing>
  seller_info {}
  payment_types {}
  shipping_info {}
  buyer_protection_info {}
  auction_info {}
  bid_history {}
  item_info {}
</listing>
<listing>
  seller_info {}
  payment_types {}
  shipping_info {}
  buyer_protection_info {}
  auction_info {}
  bid_history {}
  item_info {}
</listing>

Number of nodes: 5


### Q2 - Make a DataFrame
That is pretty nice XML. Write a look at will create a DataFrame from that.

In [6]:
# Make a Data Frame from the listing XML


## Using Recursion to Parse

Did you use some for loops to create your DataFrame, above?  

You had to write some very specific code to do that. It is "closely coupled" to the format of the XML file. What if you could flatten an XML file in a more general way?

You can do that using recursion. The set of functions below walk the XML tree to flatten it using recursion.

In [7]:
# Adapted from: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

def parse_root(root):
    return [parse_element(child) for child in root.getchildren()]

def parse_element(element, parsed=None):
    if parsed is None:
        parsed = dict()
    for key in element.keys():
        parsed[key] = element.attrib.get(key)
    if element.text:
        parsed[element.tag] = element.text
    for child in element.getchildren():
        parse_element(child, parsed)
    return parsed

def process_data(root):
    structure_data = parse_root(root)
    return pd.DataFrame(structure_data)

In [8]:
process_data(ebay.getroot())

,auction_info,bid_history,bid_increment,bidder_name,bidder_rating,brand,buyer_protection_info,closed,cpu,current_bid,...,num_items,opened,payment_types,quantity,seller_info,seller_name,seller_rating,shipping_info,started_at,time_left
0,\n,\n,,gosha555@excite.com,-2,,\n,Dec-02-00 04:57:50 PST,Pentium III 933 System,$620.00,...,1,Nov-27-00 04:57:50 PST,"Visa/MasterCard, Money Order/Cashiers Checks, ...",1,\n,cubsfantony,848,"Buyer pays fixed shipping charges, Will ship t...",$1.00,"4 days, 14 hours +"
1,\n,\n,,petitjc@yahoo.com,0,CyberTech,\n,Nov-29-00 20:54:15 PST,Intel Pentium III 800EB-MHz Coppermine CPU,$620.00,...,1,Nov-26-00 20:54:15 PST,"Visa/MasterCard, Discover, Money Order/Cashier...",1,\n,ct-inc,403,"Buyer pays actual shipping charges, Will ship ...",$0.01,"2 days, 6 hours +"
2,\n,\n,,hsclm9@peganet.com,0,CyberTech,\n,Nov-29-00 20:50:33 PST,Intel Pentium III 933EB-MHz Coppermine CPU,"$1,025.00",...,1,Nov-26-00 20:50:33 PST,"Money Order/Cashiers Checks, Personal Checks, ...",1,\n,ct-inc,403,"Buyer pays actual shipping charges, Will ship ...",$0.01,"2 days, 6 hours +"
3,\n,\n,,wizbang4,5,,\n,Nov-29-00 19:59:42 PST,Genuine Intel Pentium III 1000MHz Processor,$610.00,...,1,Nov-26-00 19:59:42 PST,"Visa/MasterCard, American Express, Money Order...",1,\n,bestbuys4systems,28,"Buyer pays fixed shipping charges, Will ship t...",$100.00,"2 days, 5 hours +"
4,\n,\n,,chul2@mail.utexas.edu,0,,\n,Nov-29-00 19:56:06 PST,INTEL Pentium III 800MHz,$535.00,...,1,Nov-26-00 19:56:06 PST,"Visa/MasterCard, Discover, Money Order/Cashier...",1,\n,sales@ctgcom.com,219,"Buyer pays fixed shipping charges, Will ship t...",$0.01,"2 days, 5 hours +"


Isn't that really nice!

There has to be a catch because I've been making you loop through code to build DataFrames and didn't just give you that code. Also, there isn't a read_xml() function in Pandas, and if there would be a candidate, you'd think that would be it.

Yup, there is a catch. Look at these NSF XML files:

In [9]:
nsf_funding = etree.parse('NSFFundingRateHistory.xml')
nsf_budget = etree.parse('NSFBudgetHistory.xml')

In [10]:
funding_df = process_data(nsf_funding.getroot())
budget_df = process_data(nsf_budget.getroot())

In [11]:
funding_df.head()

,ABBR,ACTIONS,AWARDS,CODE,FILTER,FUNDING_RATES,FY2004,FY2005,FY2006,FY2007,...,FY2012,FY2013,FY2014,FY2015,NAME,ORGUNIT,ORG_CODE,RATE,TEXT,YEAR
0,BIO,1445,257,01,RESEARCH GRANTS,\n,\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MCB,DIR,08000000,18%,DEFAULT CHART,2004
1,BIO,1749,230,01,RESEARCH GRANTS,\n,NaN,\n,NaN,NaN,...,NaN,NaN,NaN,NaN,MCB,DIR,08000000,13%,DEFAULT CHART,2005
2,BIO,1606,230,01,RESEARCH GRANTS,\n,NaN,NaN,\n,NaN,...,NaN,NaN,NaN,NaN,MCB,DIR,08000000,14%,DEFAULT CHART,2006
3,BIO,1698,235,01,RESEARCH GRANTS,\n,NaN,NaN,NaN,\n,...,NaN,NaN,NaN,NaN,MCB,DIR,08000000,14%,DEFAULT CHART,2007
4,BIO,1387,207,01,RESEARCH GRANTS,\n,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,MCB,DIR,08000000,15%,DEFAULT CHART,2008


In [12]:
budget_df.head()

,AOAM,ARI,ConstantDollars,CurrentDollars,EHR,FY1951,MREFC,NSB,NSFTotal,OIG,RRA
0,$0.98,$0.00,\n,\n,$0.00,\n,$0.00,$0.00,$1.18,$0.00,$0.20
1,$3.99,$0.00,,,$11.58,NaN,$0.00,$0.00,$26.13,$0.00,$10.55
2,$6.47,$0.00,,,$10.42,NaN,$0.00,$0.00,$32.70,$0.00,$15.81
3,$11.29,$0.00,,,$13.80,NaN,$0.00,$0.00,$58.08,$0.00,$33.00
4,$11.19,$0.00,,,$15.07,NaN,$0.00,$0.00,$90.44,$0.00,$64.18


Completely useless for analysis. There are missing colummns, rows are laid out as columns, basically it is a total mess.

So why?

You really have to look at the XML from that.

In [13]:
nsf_budget.getroot().tag

'NSFFundingMillions'

In [14]:
i = 0
budget = []
for fiscal_year in nsf_budget.getroot().getchildren():
    row = {}
    tag = fiscal_year.tag[:2]
    val = fiscal_year.tag[2:6]
    row[tag] = val
    for dollars in fiscal_year.getchildren():
        #print(dollars.tag, dollars.attrib, dollars.text)
        key_label = dollars.tag
        for child in dollars.getchildren():
            #print(child.tag, child.attrib, child.text)
            row[key_label + '_' + child.tag] = child.text
    budget.append(row)

pd.DataFrame(budget)['FY'].max()


'2015'

## Q3 - Make a DataFrame from the NSF Funding Rate History

Write the code to parse that XML into a DataFrame